Investigate how well the final report unit works.
- Retrieve T2ElasticcReport output (possibly together with redshift and ndet.
- Select one of the elasticc submission channel.
- Find out which simulation classes this corresponded to.
- For channels correspond to these:
- Study the fraction of all the different models. Possibly also do this as a function of
number of detections and/or redshift.

In [ ]:
import pymongo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from extcats import CatalogQuery
from scipy.stats import binned_statistic

In [ ]:
client = pymongo.MongoClient()

In [ ]:
db = client.ElasticcReportValidation

In [ ]:
col = db.t2

In [ ]:
# What do we need?
# df_det['ndet'][(df_det['link']==t2info['link']) & (df_det['stock']==b['stock'])]
det = []
for t2info in col.find({'unit':'T2TabulatorRiseDecline'}):
    b = {}
    b['stock'] = t2info['stock']
    b['link'] = t2info['link']
    b['ndet'] = t2info['body'][-1]['ndet']
    b['channel'] = t2info['channel']
    if b['ndet'] == 0:
        b['alldet'] = t2info['body'][-1]['alldet']
        b['nnegdet'] = t2info['body'][-1]['nnegdet']
    det.append(b)

In [ ]:
df_det = pd.DataFrame.from_dict(det)

In [ ]:
df_det.shape

In [ ]:
reports = []

In [ ]:
for t2info in col.find({'unit':'T2ElasticcReport'}):
    b = {}
    b['stock'] = t2info['stock']
    b['link'] = t2info['link']
    # ndet
    b['ndet'] = int( df_det['ndet'][(df_det['link']==t2info['link']) & (df_det['stock']==b['stock'])] )
    if b['ndet']==0:
        b['alldet'] = int( df_det['alldet'][(df_det['link']==t2info['link']) & (df_det['stock']==b['stock'])] )
        b['nnegdet'] = int( df_det['nnegdet'][(df_det['link']==t2info['link']) & (df_det['stock']==b['stock'])] )
        
    b['channel'] = t2info['channel'][0]
    if 'body' not in t2info.keys():
        print('Failed run - enough to care about?')
        print(t2info)
        continue
    if 'classifications' not in t2info['body'][-1]['report']:
        print('No classification?')
        print(t2info)
        continue
    for classification in t2info['body'][-1]['report']['classifications']:
        b[ classification['classId'] ] = classification['probability']
    reports.append( b )

In [ ]:
df_rep = pd.DataFrame.from_dict(reports)

In [ ]:
df_rep.shape

In [ ]:
df_rep['channel'].unique()

In [ ]:
modelmap = {'agn':'AGN', 'cart':'CART','ulenssinglepylima':'uLens', 'slsnihost':'SLSN', 
 'dwarfnova':'dwarf-nova', 'ulensbinary':'uLens', 'slsninohost':'SLSN', 'eb':'EB',
    'snia91bg':'SNIa91bg', 'ulenssinglegenlens':'uLens', 'sniasalt2':'SNIa', 'ilot':'ILOT', 
 'sniax':'SNIax', 'tde':'TDE', 'knb19':'KN',
       'sniibhostxtv19':'SNII', 'knk17':'KN', 'snibhostxtv19':'SNIbc', 
 'mdwarfflare':'Mdwarf-flare', 'pisn':'PISN',
       'sniitemplates':'SNII', 'rrl':'RRL', 'snibtemplates':'SNIbc', 'snicblhostxtv19':'SNIbc',
       'snichostxtv19':'SNIbc', 'snictemplates':'SNIbc', 'sniihostxtv19':'SNII',
       'sniinhostxtv19':'SNII', 'sniinmf':'SNII', 'sniinmosfit':'SNII', 'dsct':'DSC', 
 'cepheid':'Cepheid'}

In [ ]:
df_rep['model'] = df_rep['channel'].map(modelmap)

In [ ]:
df_rep

First question - for each model, how many objects and what fraction of these have redshifts?

In [ ]:
df_rep = df_rep.fillna(value=0)

In [ ]:
# Store for comparison after updating T2ElasticcReport
#df_rep.to_csv('df_ref_ereportm2.csv')
# Store second version, with only small updates to T2ElasticcReport.
#df_rep.to_csv('df_ref_ereportm3.csv')
# Store third version, where features calc based on limited set of obs + original Report.
#df_rep.to_csv('df_ref_ereportm4.csv')
# Store version where featurecalc limited at 20 ndet + original Report.
# df_rep.to_csv('df_ref_ereportm5.csv')
# Recalculated after adding treatment of negative detections and allowing z to be None
# df_rep.to_csv('df_ref_ereportm6.csv')
# Recalculated after adding priors
df_rep.to_csv('df_ref_ereportm9.csv')


In [ ]:
def getmeanstd(ndet, prob, bins):
    foo = binned_statistic(ndet, prob, statistic='mean',bins=bins)
    means = foo.statistic
    foo = binned_statistic(ndet, prob, statistic='std',bins=bins)
    stds = foo.statistic
    return means, stds

In [ ]:
report_taxonomy = {
    'SLSN':  131,
    'SNII':  113,
    'SNIa':  111,
#    'Ia':  111,
    'SNIbc': 112,
    'TDE':  132,
    'CART': 134,
    'ILOT': 133,
    'Mdwarf-flare': 122,
    'PISN': 135,
    'KN': 121,
    'SLSN-I': 131,
#    '91bg': 115,
    'SNIa91bg': 115,
    'SNIax': 114,
#    'Iax': 114,
    'dwarf-nova': 123,
    'uLens': 124,
    'Cepheid': 211,
    'EB': 214,
    'RRL': 212,
    'DSC': 213,
#    'LPV':215,  not in simulation
    'AGN':221,
#    'AGB':221,
    }

Next stage: Try to evalute one of the models.

In [ ]:
model = 'SLSN'

In [ ]:
modelid = report_taxonomy[model]

In [ ]:
df_mod = df_rep[df_rep['model']==model]

In [ ]:
ndet = df_mod['ndet']

In [ ]:
# Get the full specific model
try:
    pfull = df_mod[modelid]
except KeyError:
    pfull = np.zeros(len(ndet))

In [ ]:
# Get a list of columns corresponding to the next stage


In [ ]:
try:
    s = list( set([d for d in report_taxonomy.values() if str(d)[0:2]==str(modelid)[0:2]]) )
    pgroup = df_mod[s].sum(axis=1)
except KeyError:
    print('recurring?')
    s = list( set([d for d in [21,22] if str(d)[0:2]==str(modelid)[0:2]]) )    
    pgroup = df_mod[s].sum(axis=1)

In [ ]:
s

In [ ]:
try:
    s = list( set([d for d in report_taxonomy.values() if str(d)[0]==str(modelid)[0]]) )
    pside = df_mod[s].sum(axis=1)
except KeyError:
    print('Recurring?')
    s = list( set([d for d in [21,22] if str(d)[0]==str(modelid)[0]]) )
    pside = df_mod[s].sum(axis=1)
    

In [ ]:
s

In [ ]:
#  Find the most likely model
bestmodel = [21,  22, 1, 134,       133,       121,       122,       135,
             131, 113, 111, 115, 114, 112,
            132,       123,       124, 0]

In [ ]:
bestmod_id = df_mod[bestmodel].idxmax(axis=1)

In [ ]:
if modelid<200:
    pbestmod = (bestmod_id==modelid)
elif modelid<220:
    pbestmod = (bestmod_id==21)
else:
    pbestmod = (bestmod_id==22)

In [ ]:
pnone = (df_mod[0]==1)

In [ ]:
# Fraction of none replies
sum(pnone) / len(pnone)

In [ ]:
# Cut all rows where no answer given?
#ndet = ndet[~pnone]
#pfull = pfull[~pnone]
#pgroup = pgroup[~pnone]
#pside = pside[~pnone]
#pbestmod = pbestmod[~pnone]
#pnone = pnone[~pnone]


We now have four arrays to work with: ndet, pfull, pgroup, pside, pnone. 
Let us see how to plot this...

First version is to plot the _average_ of all classifications. Other version might be to check which fraction got more than half prob correct.

In [ ]:
bins = np.quantile(ndet,np.arange(0,1.01,0.05))
#bins = np.quantile(ndet,np.arange(0,1.01,0.1))
bins= np.unique(bins)
plotbins = (bins[0:-1] + bins[1:]) / 2

In [ ]:
mside, sside = getmeanstd(ndet, pside, bins)
mgroup, sgroup = getmeanstd(ndet, pgroup, bins)
mfull, sfull = getmeanstd(ndet, pfull, bins)
mbest, sbest = getmeanstd(ndet, pbestmod, bins)

In [ ]:
try:
    plt.plot(plotbins, mfull, '-o', label=model)
except ValueError:
    print('No single model run?')
plt.plot(plotbins, mgroup, '-o', label='Category')
plt.plot(plotbins, mside, '-o', label='Recurring?')
plt.plot(plotbins, mbest, '-o', label='IsBestchoice')


plt.ylabel('Probability')
plt.xlabel('Nbr detections')
plt.legend(loc='best')
plt.savefig('/home/jnordin/tmp/'+model+'.pdf',dpi=200)

Now look at the cases with null detections - can we say something about that?

In [ ]:
df_zero = df_rep[df_rep['ndet']==0].copy()

In [ ]:
df_zero['nfrac'] = df_zero['nnegdet'] / df_zero['alldet']

In [ ]:
plt.figure()
for model in df_zero['model'].unique():
    dfs = df_zero[(df_zero['model']==model) & (df_zero['nnegdet']>0) & (df_zero['nnegdet']>30)]
    print(model, len(dfs['alldet']))
    if len(dfs['alldet'])<10:
        continue
    
    plt.plot(dfs['alldet'],dfs['nnegdet'],'o',label=model)

plt.xlabel('All detections')
plt.ylabel('Neg detections')
plt.legend(loc='best')
    
#df_zero[ ['model','alldet', 'nnegdet']]

The vast majority of events with one more negative significant detections are EBs, uLens or AGN. One could say:
    - >1 neg detection + z>0 : AGN
    - >30 neg detections +z=0: uLens
    - Otherwise uLens or EB

In [ ]:
plt.figure(figsize=(12,10))
for model in df_zero['model'].unique():
    dfs = df_zero[(df_zero['model']==model) & (df_zero['nnegdet']>-1) & (df_zero['nfrac']<0.2)]
#    if len(dfs['alldet'])<10:
#        continue
    print(model, len(dfs['alldet']))
    
    plt.plot(dfs['alldet'],dfs['nfrac'],'o',label=model)

plt.xlabel('All detections')
plt.ylabel('Fraction of neg detections')
plt.legend(loc='best')

Or, probably better:
    - >0 neg detection + z>0 : AGN
    - >0 neg detection, z=0, frac_neg>0.2: uLens
    - >0 neg detection, z=0, frac_neg<0.2: periodic
    

How could this be implemented? Would require output from both redshift sampler and T2RiseDecline to be available. Cheat solution would be to add this in T2Xgb? Ugly but fastest...